In [1]:
import sys
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
df = pd.read_csv('fer2013.csv')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [4]:
df.head

<bound method NDFrame.head of        emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 colum

In [5]:
X_train,y_train,X_test,y_test=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [6]:
X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(28709, 2304)
(3589, 2304)
(28709,)
(3589,)


In [7]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

In [8]:
X_train = X_train.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)
print(X_train.shape)
print(X_test.shape)

(28709, 48, 48, 1)
(3589, 48, 48, 1)


In [9]:
y_train=np_utils.to_categorical(y_train,num_classes=7)
y_test=np_utils.to_categorical(y_test,num_classes=7)
print(y_train.shape)
print(y_test.shape)

(28709, 7)
(3589, 7)


In [10]:
model = Sequential()

model.add(Conv2D(32,(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))

model.add(Dense(7, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)        

In [11]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=30,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/30
28709/28709 [==============================] - 104s 4ms/step - loss: 1.6678 - accuracy: 0.3345 - val_loss: 1.5414 - val_accuracy: 0.4338
Epoch 2/30
28709/28709 [==============================] - 102s 4ms/step - loss: 1.4530 - accuracy: 0.4386 - val_loss: 1.3665 - val_accuracy: 0.4645
Epoch 3/30
28709/28709 [==============================] - 102s 4ms/step - loss: 1.3506 - accuracy: 0.4829 - val_loss: 1.2868 - val_accuracy: 0.5110
Epoch 4/30
28709/28709 [==============================] - 109s 4ms/step - loss: 1.2788 - accuracy: 0.5142 - val_loss: 1.2223 - val_accuracy: 0.5344
Epoch 5/30
28709/28709 [==============================] - 113s 4ms/step - loss: 1.2274 - accuracy: 0.5335 - val_loss: 1.2004 - val_accuracy: 0.5364
Epoch 6/30
28709/28709 [==============================] - 123s 4ms/step - loss: 1.1802 - accuracy: 0.5488 - val_loss: 1.1543 - val_accuracy: 0.5584
Epoch 7/30
28709/28709 [==============================] - 103s 

In [13]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")